In [1]:
search_for = 138
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.12465262413024902
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 1097396311
type: [1, 1, 1, 1, 138] 138
cases of this type: 362673936
100000 55.53592627774841
200000 115.26281613709249
300000 128.73803637648933
400000 135.17235477046407
500000 129.7359560061933
600000 121.20754897609251
700000 136.1374264043689
800000 124.99321311429284
900000 123.35345747152024
1000000 127.88099311844748
1100000 122.11292102922627
1200000 125.51665491702644
1300000 105.4004803288971
1400000 51.231759203691176
1500000 113.00863870314544
1600000 104.0458369941809
1700000 104.07713301554323
1800000 106.7048998294971
1900000 101.1556427955982
2000000 112.77839915963091
2100000 102.29005383949246
2200000 107.5257195615434
2300000 108.39531733744977
2400000 101.65042022087896
2500000 110.23920626073661
2600000 97.69923414562707
2700000 24.609766007710338
2800000 104.29116860616627
2900000 101.88822411318988
300000

29300000 106.9393009192504
29400000 106.97367788943937
29500000 107.6555627573241
29600000 103.49278389203917
29700000 106.541804655081
29800000 106.4040495258603
29900000 106.08112838725486
30000000 103.45305870838749
30100000 101.11828482375138
30200000 91.62928731766664
30300000 41.78406465240305
30400000 102.21814741493858
30500000 102.70644454614092
30600000 102.7306131500515
30700000 105.48436606871522
30800000 102.0185505707251
30900000 101.7586367321184
31000000 106.25429954925407
31100000 108.75418747671776
31200000 105.11248249408456
31300000 101.07967215108235
31400000 44.990574723160734
31500000 108.23891874428485
31600000 110.4176850689213
31700000 111.0089581944446
31800000 51.68902670368491
31900000 94.23035881841503
32000000 109.31867707834408
32100000 109.37035787894133
32200000 110.36384238085289
32300000 106.82734444974886
32400000 104.56535670916378
32500000 107.91750387278529
32600000 105.17555184258437
32700000 106.45299166633329
32800000 103.31604789758691
329000

59100000 93.0695869881133
59200000 23.09560289399855
59300000 98.60032169739088
59400000 98.1298263520307
59500000 96.47151185200346
59600000 98.26194178048415
59700000 100.92142770647659
59800000 99.90806281957677
59900000 102.32845129887892
60000000 99.4663832231764
60100000 43.69739918943234
60200000 105.79053087738008
60300000 105.15832389399714
60400000 105.25505403196189
60500000 103.8090958554131
60600000 104.57471427997217
60700000 103.8030879056346
60800000 102.20166929541969
60900000 64.73631955219808
61000000 77.8416716021053
61100000 100.35563083566103
61200000 97.80761587968786
61300000 97.43142963254148
61400000 103.17249061282624
61500000 94.00016672800069
61600000 91.37376360626008
61700000 92.46286761845376
61800000 33.11873552039376
61900000 87.8878449118571
62000000 91.32707239662294
62100000 92.18921963039612
62200000 96.59298772140514
62300000 93.13162529621675
62400000 95.8359341717975
62500000 98.37967522883903
62600000 103.53855410077509
62700000 35.716302521949

89100000 106.72642216237824
89200000 101.4874927139083
89300000 108.90977058291003
89400000 113.30025931264512
89500000 107.06618646930757
89600000 107.71425943068753
89700000 108.76502674501195
89800000 103.60241104852393
89900000 106.64293323093595
90000000 75.41334085960239
90100000 61.96753047200297
90200000 92.6671844912518
90300000 96.57699839986259
90400000 93.10315275562942
90500000 97.88419534588529
90600000 98.03604884991418
90700000 39.53754608211025
90800000 79.09040774364904
90900000 100.19953820776855
91000000 95.67067952381943
91100000 93.7593368395455
91200000 98.48426909513269
91300000 92.18239349864874
91400000 34.636559440894004
91500000 107.41480081698428
91600000 105.34759963946283
91700000 102.62376732634345
91800000 107.23495630824632
91900000 109.35551860974225
92000000 108.74457559078678
92100000 107.3319946299165
92200000 102.92563638355547
92300000 101.83722473189624
92400000 103.23038403486038
92500000 96.28732081242529
92600000 104.01582668539297
92700000 3

118600000 101.07112861451249
118700000 97.17166439056125
118800000 101.25426594765622
118900000 94.58475587886679
119000000 87.70534722760765
119100000 85.9938818296447
119200000 38.7931273306121
119300000 91.32324077445911
119400000 89.10241442579421
119500000 92.62588072785664
119600000 45.65246380661491
119700000 68.97225432375343
119800000 92.02416830506291
119900000 91.62255092259686
120000000 90.99347477962418
120100000 34.31213702721937
120200000 94.70532220163707
120300000 88.34160843472661
120400000 94.19070964737793
120500000 99.26482045645017
120600000 91.82829201582247
120700000 103.80590655837892
120800000 94.86459518728817
120900000 103.96219657596099
121000000 100.84669147506268
121100000 97.73569405152759
121200000 105.1359899023492
121300000 102.50475252130967
121400000 94.33762198327722
121500000 97.95747360089537
121600000 89.17453170368658
121700000 94.84652724082008
121800000 43.00019917353048
121900000 80.40373383417807
122000000 87.5192353155915
122100000 94.2808

...

982300000 1.0636615100272657
982400000 1.0552386649700358
982500000 1.063821529647673
982600000 1.1491651360457837
982700000 4.063083173417698
982800000 7.490162698504377
982900000 7.526767494297971
983000000 3.951880384556335
983100000 5.091561876101687
983200000 7.38697688089006
983300000 7.763802898376512
983400000 6.821452920464569
983500000 6.1083307827077435
983600000 5.68016771136179
983700000 1.8491575029665874
983800000 5.582611467729099
983900000 5.944485049718725
984000000 6.55829968882965
984100000 5.91879611776458
984200000 5.878413192031722
984300000 3.461913780512425
984400000 2.9549962528172036
984500000 5.879854100302161
984600000 5.763073550723815
984700000 6.6506385834183614
984800000 5.951693769380372
984900000 5.428211456926472
985000000 3.0098736858916935
985100000 4.6492640745070135
985200000 5.902681801791096
985300000 6.558166613869902
985400000 6.309375471167368
985500000 5.979314078802152
985600000 4.2735087334124024
985700000 2.803460458069808
985800000 5.80

1010700000 1.7065850888935137
1010800000 2.700751803577383
1010900000 2.624074895778411
1011000000 3.3091392906861636
1011100000 2.8114580430992424
1011200000 2.3275281181534555
1011300000 0.8950167438620754
1011400000 2.905142130077137
1011500000 2.770183139761028
1011600000 3.3210432713484304
1011700000 2.9651993072748084
1011800000 2.9912474296826
1011900000 1.560113956138502
1012000000 1.7348275355629388
1012100000 2.9180476191752596
1012200000 2.975576901160208
1012300000 3.1403688365716755
1012400000 2.5767090858831425
1012500000 2.3601598148632235
1012600000 0.8879148313520789
1012700000 2.1532957571585576
1012800000 2.603637868374275
1012900000 2.8102928341248927
1013000000 2.6132774746634184
1013100000 2.497664417294575
1013200000 1.4939190723476687
1013300000 1.1419124817727346
1013400000 2.4956167429875333
1013500000 2.3949938984108385
1013600000 2.794158018136795
1013700000 2.2956843753890337
1013800000 2.257396377711886
1013900000 0.8819218243206653
1014000000 1.7223957872

1036600000 2.3972099607722557
1036700000 3.0360201508320235
1036800000 4.262597559575464
1036900000 2.208395264082386
1037000000 3.1712504112457753
1037100000 3.9827070921786363
1037200000 2.6827118759860755
1037300000 3.2106269943623054
1037400000 3.673545227529835
1037500000 2.9719554963060375
1037600000 3.2601595660580176
1037700000 3.514589461517548
1037800000 2.8624070834289967
1037900000 3.455522161251581
1038000000 3.176897062235922
1038100000 2.71483462740708
1038200000 3.2881735061815447
1038300000 3.213569858329266
1038400000 2.829102438086202
1038500000 3.4386214470343397
1038600000 3.342648561342422
1038700000 2.8230424864053627
1038800000 3.3804239293600413
1038900000 3.480095731412807
1039000000 3.0637815087748175
1039100000 3.126339060523201
1039200000 3.734552222327927
1039300000 3.0819468605463007
1039400000 2.573962659354578
1039500000 3.897132218049299
1039600000 3.42987780831961
1039700000 2.4388046549268503
1039800000 3.5068646595259185
1039900000 3.60572878814317


1057800000 2.0729049002456885
1057900000 2.0739509742522655
1058000000 1.8091645118323836
1058100000 1.9628745074842824
1058200000 2.4202745756896915
1058300000 1.80498937867955
1058400000 1.6299063744874585
1058500000 2.2169025269545855
1058600000 1.921677795757633
1058700000 1.7459064939600935
1058800000 1.9652756794523658
1058900000 1.985227870607918
1059000000 1.722514458070498
1059100000 1.9312816382984836
1059200000 2.3641023968016355
1059300000 1.5216511664900714
1059400000 1.779904663950787
1059500000 2.347524102919212
1059600000 1.852448715591086
1059700000 1.5243184004654171
1059800000 2.211243282738208
1059900000 1.8412189968473998
1060000000 1.6558155236384013
1060100000 1.852407788054712
1060200000 2.1292719395468116
1060300000 1.3809823598707158
1060400000 1.7698133713920394
1060500000 2.1668810134379273
1060600000 1.6480358206736805
1060700000 1.4478289110597808
1060800000 2.069745023882355
1060900000 1.7328879566052975
1061000000 1.3787178646018223
1061100000 2.04704827

1084500000 0.10683039127357866
1084600000 0.060288117678288615
1084700000 0.05937532795251144
best so far: 0
type: [2, 1, 3, 1, 23] 138
cases of this type: 2518569
1084800000 0.14523835187328724
1084900000 0.3440478470057428
1085000000 0.2062334211165713
1085100000 0.17966703315913216
1085200000 0.17903063597438734
1085300000 0.19796254169096747
1085400000 0.24952219344963167
1085500000 0.28573136750266154
1085600000 0.2169396029495113
1085700000 0.18839005184800225
1085800000 0.11689352316854597
1085900000 0.07815361780269213
1086000000 0.06547858785691395
1086100000 0.08798647120875121
1086200000 0.06321431600977116
1086300000 0.09185240771280595
1086400000 0.2005434018611133
1086500000 0.17084311389548712
1086600000 0.22107866525602937
1086700000 0.1865734286777973
1086800000 0.23961716183464593
1086900000 0.16971838739447528
1087000000 0.15260096447829805
1087100000 0.2100659270348317
1087200000 0.16805252762864165
best so far: 0
type: [2, 1, 3, 23, 1] 138
cases of this type: 10950